In [1]:
import nltk
import random
import os
import json
from nltk.stem import  PorterStemmer
import random
import pickle

In [27]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                #files_f.remove('neutral')
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    dataset+=[(sentence,directory[-1:]) for sentence in data_temp]
    return dataset

In [20]:
#initialising porterstemmer
ps=PorterStemmer()

In [21]:
def to_unigram(data):
    dataset=[]
    dataset+=[([items for items in sentence.split()],aspect) for sentence,aspect in data]    
    return dataset    

In [28]:
#execution
labeled_data=load_dataset()    #loading labeled sentences
random.shuffle(labeled_data)
labeled_data=to_unigram(labeled_data)    #converting to unigram

In [31]:
#loading vocab
with open('../NaiveBayes/vocab') as f:
    vocab=json.load(f)

In [29]:
#bigrams
print labeled_data[100]

([u'degree', u'work', u'forced', u'put'], '3')


In [30]:
print len(labeled_data)

179644


In [33]:
print len(vocab)
print vocab[:10]

307
[u'ridicul', u'lack', u'poorli', u'bright', u'unorgan', u'abil', u'rough', u'hate', u'enthusiast', u'slow']


In [34]:
def extract_features(document):       #features are bag of words. document is a list of words of a sentence 
    features = {}
    document= [ps.stem(word) for word in document]
    for word in vocab:
        features['contains(%s)' % word] = (word in document)
    return features        

In [35]:
train_test = nltk.classify.apply_features(extract_features, labeled_data)

In [36]:
#dividing into training and test data
train_set=train_test[:-10000]
test_set=train_test[-10000:]

In [37]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
with open('naive_bayes_unigram_model','w') as f:
    pickle.dump(classifier,f)

In [38]:
accuracy=nltk.classify.accuracy(classifier, test_set)
with open('accuracy_unigram_model','w') as f:
    json.dump(accuracy,f)

In [39]:
print 'Accuracy : '+ str(accuracy)

Accuracy : 0.4929


In [40]:
#sample
sen='good learning experience'
classifier.classify(extract_features(sen.split()))

'3'